In [40]:
import pandas as pd
import numpy as np
import missingno as msno
from sklearn import tree
from scipy.io import arff

In [41]:
df = arff.loadarff("dataset/arrhythmia.arff")
df = pd.DataFrame(df[0])
df

,age,sex,height,weight,QRSduration,PRinterval,Q-Tinterval,Tinterval,Pinterval,QRS,...,chV6_QwaveAmp,chV6_RwaveAmp,chV6_SwaveAmp,chV6_RPwaveAmp,chV6_SPwaveAmp,chV6_PwaveAmp,chV6_TwaveAmp,chV6_QRSA,chV6_QRSTA,binaryClass
0,75.0,b'0',190.0,80.0,91.0,193.0,371.0,174.0,121.0,-16.0,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,b'N'
1,56.0,b'1',165.0,64.0,81.0,174.0,401.0,149.0,39.0,25.0,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,b'N'
2,54.0,b'0',172.0,95.0,138.0,163.0,386.0,185.0,102.0,96.0,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,b'N'
3,55.0,b'0',175.0,94.0,100.0,202.0,380.0,179.0,143.0,28.0,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,b'P'
4,75.0,b'0',190.0,80.0,88.0,181.0,360.0,177.0,103.0,-16.0,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,b'N'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,53.0,b'1',160.0,70.0,80.0,199.0,382.0,154.0,117.0,-37.0,...,0.0,4.3,-5.0,0.0,0.0,0.7,0.6,-4.4,-0.5,b'P'
448,37.0,b'0',190.0,85.0,100.0,137.0,361.0,201.0,73.0,86.0,...,0.0,15.6,-1.6,0.0,0.0,0.4,2.4,38.0,62.4,b'N'
449,36.0,b'0',166.0,68.0,108.0,176.0,365.0,194.0,116.0,-85.0,...,0.0,16.3,-28.6,0.0,0.0,1.5,1.0,-44.2,-33.2,b'N'
450,32.0,b'1',155.0,55.0,93.0,106.0,386.0,218.0,63.0,54.0,...,-0.4,12.0,-0.7,0.0,0.0,0.5,2.4,25.0,46.6,b'P'


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Columns: 280 entries, age to binaryClass
dtypes: float64(206), object(74)
memory usage: 988.9+ KB


In [43]:
pd.DataFrame(df.isnull().sum()).sort_values(by=0, ascending=False).head(10)

,0
J,376
P,22
T,8
QRST,1
heartrate,1
chAVR_SwaveAmp,0
chAVR_RwaveAmp,0
chAVR_QwaveAmp,0
chAVR_JJwaveAmp,0
chDIII_QRSTA,0


In [44]:
df.drop("J",axis=1,inplace=True)
df.dropna(axis=0, inplace=True)

In [45]:
pd.DataFrame(df.isnull().sum()).sort_values(by=0, ascending=False).head(10)

,0
age,0
chDIII_PwaveAmp,0
chAVR_RwaveAmp,0
chAVR_QwaveAmp,0
chAVR_JJwaveAmp,0
chDIII_QRSTA,0
chDIII_QRSA,0
chDIII_TwaveAmp,0
chDIII_SPwaveAmp,0
chDII_TwaveAmp,0
